If you haven't covered langmem and basic openai-agents-sdk go through first:

1. [Understand LangMem Core APIs](https://colab.research.google.com/drive/1YJNrnQRMgeNTigIuWOfykt-Z5L_DDmsa?usp=sharing)

# **OpenAI Agents SDK with LangMem Memory Tools & InMemory Store**

In [1]:
!pip install -Uq openai-agents===0.0.7 langmem langchain-google-genai langmem-adapter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [3]:
from google.colab import userdata
import os

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL = "gemini-2.0-flash"

# import os
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# MODEL = "gpt-4o"

In [4]:
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel, set_tracing_disabled

client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url=BASE_URL
)

model = OpenAIChatCompletionsModel(model=MODEL, openai_client=client)

set_tracing_disabled(disabled=False)

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langgraph.store.memory import InMemoryStore

import asyncio
from agents import Agent, Runner

os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

store = InMemoryStore(
      index={
          "dims": 768,
          "embed": GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"),
          # "dims": 1536,
          # "embed": "openai:text-embedding-3-small",
      }
    )


In [6]:

namespace=("assistant", "collection")
print(store.search(namespace))

[]


## 2. OpenAI Agents SDK with Memory Layer

**Quick Note on langmem_adapter:** We can not directly use langchain tool with openai agents sdk. There is difference in how to invoke the tool and much more. Additionally when using completions api with gemini we can not have uuid as argument type for a function used as tool.

To workaround all these tools I created a simple class that takes langmem tools and make them usable with OpenAI Agents SDK. Next I extended it to use langmem features like:
-  Have Dynamic NameSpace Tuples so we can pass userId etc. in runtime using OpenAI Agents SDK Context.
- Inject Store Dynamically or statically
- Handle issues like above uuid one.

You can review the source code here:

https://github.com/mjunaidca/langmem-openaiagents-adapter/blob/main/src/langmem_adapter/openai_agents_sdk.py

In [7]:
from langmem import create_manage_memory_tool, create_search_memory_tool
from langmem_adapter import LangMemOpenAIAgentToolAdapter

# Using with InMemory Store and Static NameSpace is simple as follows

adapter = LangMemOpenAIAgentToolAdapter(create_manage_memory_tool(namespace=namespace,store=store))
call_manage_memory_tool = adapter.as_tool()

search_memory_tool_adapter = LangMemOpenAIAgentToolAdapter(create_search_memory_tool(namespace=namespace,store=store))
call_search_memory_tool = search_memory_tool_adapter.as_tool()

In [8]:
# Its a FunctionTool type as in OpenAI Agetns SDK
call_search_memory_tool

FunctionTool(name='search_memory', description='Search your long-term memories for information relevant to your current context.', params_json_schema={'properties': {'query': {'title': 'Query', 'type': 'string'}, 'limit': {'title': 'Limit', 'type': 'integer'}, 'offset': {'title': 'Offset', 'type': 'integer'}, 'filter': {'default': '', 'title': 'Filter', 'type': 'string'}}, 'required': ['query', 'limit', 'offset'], 'title': 'search_memoryArgs', 'type': 'object'}, on_invoke_tool=<bound method LangMemOpenAIAgentToolAdapter._on_invoke_tool of <langmem_adapter.openai_agents_sdk.LangMemOpenAIAgentToolAdapter object at 0x79c7641d2410>>, strict_json_schema=True)

| **Note:** Here the tools are defined to demonstrate the concent. In production we will use a Persistant Store and have passed all Args with a more better design pattern

In [9]:
agent_system_prompt_memory = """
< Role >
You are Junaids executive assistant. You are a top-notch executive assistant who cares about AI Agents and performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage Junaid's communications and schedule:

1. manage_memory - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
2. search_memory - Search for any relevant information that may have been stored in memory
</ Tools >

"""

In [10]:

tools= [
    call_search_memory_tool,
    call_manage_memory_tool
]


In [11]:
import asyncio
import typing
import uuid
from agents import function_tool

agent = Agent(
    name="Assistant",
    instructions=agent_system_prompt_memory,
    model=model,
    tools=tools
)

async def run_example(message: str):

    result = await Runner.run(
        agent,
        message,
    )
    print(result.final_output)


asyncio.run(run_example("Ahmad is my friend"))


OK. I've saved that Ahmad is your friend.



In [12]:
print(store.search(namespace))

[Item(namespace=['assistant', 'collection'], key='afe75d41-9776-44a4-8b12-e1b6ba2cb551', value={'content': "Ahmad is Junaid's friend"}, created_at='2025-04-05T17:22:45.371898+00:00', updated_at='2025-04-05T17:22:45.371903+00:00', score=None)]


In [13]:
asyncio.run(run_example("Who are my friends"))

Okay, it looks like Ahmad is one of your friends.



In [15]:
asyncio.run(run_example("Oh no I meant Muhammad Ahmad not Ahmad is my Friend. Update it without asking any Qs"))

Excellent! I have updated the memory to reflect that Muhammad Ahmad is Junaid's friend.



In [16]:
asyncio.run(run_example("Who are my friends"))

Okay, it looks like Muhammad Ahmad is listed as your friend. Would you like me to add anyone else to your list of friends?



In [17]:
asyncio.run(run_example("Sir Zia Ullah Khan is my Mentor"))

OK. I've saved that Sir Zia Ullah Khan is your mentor.



In [18]:
asyncio.run(run_example("Who is my Mentor?"))

Sir Zia Ullah Khan is your mentor.



In [19]:
print(store.search(namespace))

[Item(namespace=['assistant', 'collection'], key='afe75d41-9776-44a4-8b12-e1b6ba2cb551', value={'content': "Muhammad Ahmad is Junaid's friend"}, created_at='2025-04-05T17:28:12.973047+00:00', updated_at='2025-04-05T17:28:12.973050+00:00', score=None), Item(namespace=['assistant', 'collection'], key='1eefde42-3558-4870-bbdd-7414c5603cd1', value={'content': "Sir Zia Ullah Khan is Junaid's Mentor."}, created_at='2025-04-05T17:28:40.932253+00:00', updated_at='2025-04-05T17:28:40.932257+00:00', score=None)]
